## Simulate read counts of an idealized scGRO-seq dataset for mm10 genes

In [ ]:
.libPaths("/home/mahat/.conda/envs/r422/lib/R/library")
.libPaths()

In [ ]:
suppressMessages({
    library(tidyverse)
    library(matrixStats)
    library(Seurat)
    library(BiocParallel)
    library(foreach)
    library(doParallel)
    library(plyranges)
    library(viridis)
    library(ggpointdensity)
});

In [ ]:
options(
    repr.plot.width=4,
    repr.plot.height=4,
    jupyter.plot_mimetypes = "image/svg+xml"
);
theme_set(theme_classic() +
    theme(
        axis.title.x = element_text(color="black", size=14, face="bold"), 
        axis.title.y = element_text(color="black", size=14, face="bold"),
        axis.text = element_text(color="black", size=12, face="bold"),
        plot.title = element_text(face="bold", size=14, hjust = 0.5),
        axis.line = element_blank(),
        # axis.ticks = element_blank()
        panel.border = element_rect(colour = "grey", fill=NA, linewidth=1)
    )
);

In [ ]:
# Get equation and r^2 as string
# https://groups.google.com/forum/#!topic/ggplot2/1TgH-kG5XMA

lm_eqn = function(x, y) {
    m = lm(y ~ x);
    eq = substitute(italic(y) == a + b %.% italic(x)*","~~italic(r)^2~"="~r2, 
         list(a = format(unname(coef(m)[1]), digits = 2),
              b = format(unname(coef(m)[2]), digits = 2),
             r2 = format(summary(m)$r.squared, digits = 2)))
    as.character(as.expression(eq));
}

### Set simulation parameters

In [ ]:
Ncells = 2642; # N cells

pol2_speed     = 2500; # speed of RNA Pol2 in bp / min
trimEndLength = 1000;
maxLen = 10000; # max feature length allowed

In [ ]:
# read in estimated burst frequency/size
obs_burst = read_csv("../data/scGROv2p8_max10kbp_max5kbp_burst_rate_0p10_captureEfficiency.csv");
obs_burst = GRanges(obs_burst)
names(obs_burst) = obs_burst$feature;
obs_burst$feature = NULL;
obs_burst$length = NULL;
head(obs_burst)

In [ ]:
# use real feature lengths
features = read_bed("../data/dREG_refinedFeatures_mES_mm10_OSNcustomEnhancers_SEs.bed");
features$score = NULL;

# truncate long features
longf = which( width(features) >= maxLen+trimEndLength );
features[longf] = features[longf] 

features = features %>%
    filter( width >= maxLen+trimEndLength ) %>%
    filter( substr(name, 0, 2) == "GN" ) %>%
    filter( name %in% names(obs_burst) ) %>%
    anchor_center() %>%
    mutate(width = width - trimEndLength) %>%
    resize( width = maxLen, fix="start" );
Nfeat = length(features);

In [ ]:
# set burst parameters for each feature
features = features %>%
    mutate( B.SIZE = obs_burst[name]$burst_size ) %>%
    mutate( B.RATE = obs_burst[name]$burst_rate );

features

In [ ]:
summary(width(features))

In [ ]:
gene_time = width(features) / pol2_speed;

In [ ]:
as.data.frame(features) %>%
    ggplot( aes(x=B.SIZE)  ) +
    # geom_histogram(binwidth=0.25, boundary=0) +
    geom_histogram(bins = 32, boundary=0, color = "white") +
    xlim(0, 5) +
    xlab("Observed burst size") +
    ylab("Number of features")

In [ ]:
as.data.frame(features) %>%
    ggplot( aes(x=B.RATE)  ) +
    geom_histogram(binwidth=0.1, boundary=0) +
    xlab("Observed burst rate (per hour)") +
    ylab("Number of features") +
    xlim(0, 10);

### Simulate molecules and reads
Molecules are simulated directly from burst rate and size.
Reads are simulated by randomly sampling molecules.

In [ ]:
simulate_reads = function( features, reads_per_cell ) {
    # compute amount of time visible on each gene
    # time will be in minutes from how pol2_speed is defined
    # (bp) / (bp / min) => min
    ncell = length(reads_per_cell);
    nfeat = length(features);
    
    # sample burst counts from poisson distribution
    SimPol2 = rpois(
        n = ncell*nfeat,
        lambda = features$B.RATE/60*gene_time
    );
    
    # simulate an observed burst size for this cell
    bsize = rnorm(
        n = ncell*nfeat,
        mean = features$B.SIZE,
        sd   = features$B.SIZE/4
    );
    bsize = ifelse( bsize < 1, 1, bsize );
    SimPol2 = round(SimPol2 * bsize, 0);
    SimPol2 = matrix( as.numeric(SimPol2), nrow = nfeat, ncol=ncell);

    # label columns as cells, rows as features
    colnames(SimPol2) = paste0("cell", 1:ncell);
    rownames(SimPol2) = features$name;

    # randomly sample read counts
    counts = foreach( i = 1:ncell, .combine = "cbind" ) %dopar% {
        #ftprobs  = SimPol2[,i] / sum(SimPol2[,i]);
        molecules = rep(1:nfeat, SimPol2[,i]);
        #ftcounts = sample( molecules, size=reads_per_cell[i], replace=T, prob=ftprobs );
        ftcounts = sample( molecules, size=reads_per_cell[i], replace=F );
        ftcounts = as.data.frame(table(ftcounts), stringsAsFactors=F);
        out = rep(0, nfeat);
        out[as.integer(ftcounts[,1])] = ftcounts$Freq;
        return( out );
    }
    
    colnames(counts) = paste0("cell", 1:ncell);
    rownames(counts) = features$name;

    return(list(SimPol2, counts));
}

In [ ]:
counts = readRDS( "../data/scGROv2p8_mapq3qc_max10kbp_filtered_counts.rds" );

# use observed reads per cell
rpc = colSums( counts[features$name,] );
summary(rpc)
# filter RPC to minimize differences in read counts from biology (e.g. cell cycle)
# divide by 2 since we simulated one allele, not two
sim = simulate_reads( features, rpc[rpc <= 1500] );

### Check simulation outputs

In [ ]:
sim[[1]][1:9,1:9]

In [ ]:
sim[[2]][1:9,1:9]

In [ ]:
data.frame(RPC=colSums(sim[[1]])) %>%
    ggplot( aes(x=RPC)  ) +
    geom_histogram(binwidth=50, boundary=0) +
    xlab("Active Pol2 per cell") +
    ylab("Number of cells");

In [ ]:
data.frame(RPC=colSums(sim[[2]])) %>%
    ggplot( aes(x=RPC) ) +
    geom_histogram(binwidth=25, boundary=0) +
    xlab("Sampled reads per cell") +
    ylab("Number of cells") +
    xlim(0, 2000);

### Estimate kinetic parameters from simulation

In [ ]:
estBurstSize = function(counts) {
    counts = counts[counts>0];
    return(mean(counts));
}

out = data.frame(eBSIZE=apply(sim[[1]], 1, estBurstSize));
txt=lm_eqn(features$B.SIZE, out$eBSIZE);

out %>%
    ggplot( aes(x=features$B.SIZE, y=eBSIZE) ) +
    geom_pointdensity() +
    scale_x_log10() +
    scale_y_log10() +
    scale_color_viridis() +
    annotate(geom = "label", x = 2.5, y = 12, label = txt, parse=T) +
    ggtitle("Sample efficiency = 100%") +
    xlab("True burst size") +
    ylab("Estimate");

In [ ]:
out = data.frame(
    efficiency=median(colSums(sim[[2]]))/median(colSums(sim[[1]])),
    trueSize=features$B.SIZE,
    trueFreq=features$B.RATE,
    obSize=apply(sim[[2]], 1, estBurstSize),
    estFreq=rowMeans(sim[[2]]>0)/gene_time*60,
    length=width(features)
) %>%
    na.omit %>%
    mutate(lnorm=maxLen/length) %>%
    mutate(estSize=1 + (obSize-1)/efficiency) %>%
    mutate(detectionRate = estSize*efficiency) %>%
    mutate(detectionRate = ifelse(detectionRate>1, 1, detectionRate)) %>%
    mutate(estFreq=estFreq/detectionRate);

In [ ]:
txt=lm_eqn(out$trueSize, out$estSize);

bs = out %>%
    ggplot( aes(x=trueSize, y=estSize) ) +
    geom_pointdensity() +
    scale_x_log10() +
    scale_y_log10() +
    scale_color_viridis() +
    # annotate(geom = "label", x = 3, y = 20, label = txt, parse=T) +
    annotate(geom="label", label.size=NA, x=0, y=Inf, hjust=0, vjust=1, label=txt, parse=T, fill=NA) +
    # ggtitle(paste0("Capture efficiency = ", 100*round(out$efficiency[1], 2), "%")) +
    xlab("True burst size") +
    ylab("Estimated burst size") +
    theme( legend.key.size = unit(6, 'mm'), legend.background = element_blank(), legend.position = c(0.875, 0.25), 
          legend.title = element_text(size = 0), legend.text = element_text(size = 10))
bs
ggsave(bs, filename = "../plots/Simulation_burst_size.pdf", width=4, height=4);

In [ ]:
txt=lm_eqn(out$trueFreq, out$estFreq);

bf = out %>%
    ggplot( aes(x=trueFreq, y=estFreq) ) +
    geom_pointdensity() +
    scale_x_log10() +
    scale_y_log10() +
    scale_color_viridis() +
    # annotate(geom = "label", x = 3, y = 20, label = txt, parse=T) +
    annotate(geom="label", label.size=NA, x=0, y=Inf, hjust=0, vjust=1, label=txt, parse=T, fill=NA) +
    # ggtitle(paste0("Capture efficiency = ", 100*round(out$efficiency[1], 2), "%")) +
    xlab("True burst rate (per hour)") +
    ylab("Estimated burst rate (per hour)") +
    theme( legend.key.size = unit(6, 'mm'), legend.background = element_blank(), legend.position = c(0.875, 0.25), 
          legend.title = element_text(size = 0), legend.text = element_text(size = 10))
bf
ggsave(bf, filename = "../plots/Simulation_burst_frequency.pdf", width=4, height=4);